In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt

In [2]:
data_dir = r"C:\Users\mukes\Downloads\Sign-Language-detection-main\Sign-Language-detection-main\project1\data set images"


In [ ]:
filenames = os.listdir(data_dir)
categories = []

for f_name in filenames:
    category = f_name.split('.')[0][0]
    categories.append(category)
df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})


In [4]:
df

,filename,category
0,A2389.jpg,A
1,A239.jpg,A
2,A2390.jpg,A
3,A2391.jpg,A
4,A2392.jpg,A
...,...,...
76451,Z995.jpg,Z
76452,Z996.jpg,Z
76453,Z997.jpg,Z
76454,Z998.jpg,Z


In [5]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
batch_size = 32
image_width = 128
image_height = 128
image_size = (image_width, image_height)
image_channels = 3

train_datagen = ImageDataGenerator(rotation_range=15,
                                   rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=data_dir,
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 61164 validated image filenames belonging to 26 classes.


In [7]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    directory=data_dir,
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 15292 validated image filenames belonging to 26 classes.


In [8]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, image_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(26, activation='softmax'))  # 26 letters of the alphabet


c:\Users\mukes\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
optimizer = RMSprop()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,955,098 (49.42 MB)

 Trainable params: 12,953,626 (49.41 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [12]:
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)
callbacks = [earlystop, learning_rate_reduction]

In [13]:
epochs = 16
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validate_df)//batch_size,
    steps_per_epoch=len(train_df)//batch_size,
    callbacks=callbacks
)

c:\Users\mukes\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/16
1911/1911 ━━━━━━━━━━━━━━━━━━━━ 639s 333ms/step - accuracy: 0.2737 - loss: 2.7335 - val_accuracy: 0.5092 - val_loss: 1.4904 - learning_rate: 0.0010
Epoch 2/16
   1/1911 ━━━━━━━━━━━━━━━━━━━━ 9:36 302ms/step - accuracy: 0.7188 - loss: 1.3012

c:\Users\mukes\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1911/1911 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.7188 - loss: 1.3012 - val_accuracy: 0.3789 - val_loss: 2.7422 - learning_rate: 0.0010
Epoch 3/16
1911/1911 ━━━━━━━━━━━━━━━━━━━━ 683s 357ms/step - accuracy: 0.7119 - loss: 0.8559 - val_accuracy: 0.6073 - val_loss: 1.6172 - learning_rate: 0.0010
Epoch 4/16
1911/1911 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.7500 - loss: 0.6210 - val_accuracy: 0.8733 - val_loss: 0.3587 - learning_rate: 0.0010
Epoch 5/16
1911/1911 ━━━━━━━━━━━━━━━━━━━━ 674s 353ms/step - accuracy: 0.8133 - loss: 0.5579 - val_accuracy: 0.7315 - val_loss: 0.9763 - learning_rate: 0.0010
Epoch 6/16
1911/1911 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9062 - loss: 0.3334 - val_accuracy: 0.9222 - val_loss: 0.2159 - learning_rate: 0.0010
Epoch 7/16
1911/1911 ━━━━━━━━━━━━━━━━━━━━ 675s 353ms/step - accuracy: 0.8525 - loss: 0.4352 - val_accuracy: 0.8698 - val_loss: 0.4261 - learning_rate: 0.0010
Epoch 8/16
   1/1911 ━━━━━━━━━━━━━━━━━━━━ 9:50 309ms/step - accuracy:

In [14]:
model.save("sign_language_model.h5")

In [6]:
import os
import random
import shutil

# Define paths
dataset_path = r"C:\Users\mukes\Downloads\Sign-Language-detection-main\Sign-Language-detection-main\project1\data set images"  # Original dataset
testdataset_path = r"C:\Users\mukes\Downloads\Sign-Language-detection-main\Sign-Language-detection-main\project1\testdatabase"  # New test dataset folder

# Create testdataset folder if it doesn't exist
os.makedirs(testdataset_path, exist_ok=True)

# Get all image filenames
all_images = os.listdir(dataset_path)

# Randomly select 1000 images
selected_images = random.sample(all_images, 1000)

# Copy selected images to the testdataset folder
for img in selected_images:
    src = os.path.join(dataset_path, img)
    dst = os.path.join(testdataset_path, img)
    shutil.copy(src, dst)

print("✅ 1000 images successfully copied to testdataset!")


✅ 1000 images successfully copied to testdataset!


# Testing Process..


In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the trained model
model = tf.keras.models.load_model("sign_language_model.h5")

# Define image size (same as training)
image_width, image_height = 128, 128
image_size = (image_width, image_height)

# Path to the test dataset folder
test_dir = r"C:\Users\mukes\Downloads\Sign-Language-detection-main\Sign-Language-detection-main\project1\testdatabase"

# Get the list of test filenames
test_filenames = os.listdir(test_dir)

# Creating a DataFrame for test data
test_df = pd.DataFrame({'filename': test_filenames})

# Create an ImageDataGenerator for testing (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_dir,
    x_col='filename',
    target_size=image_size,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# Get the class labels from the training data
class_labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Make predictions
predictions = model.predict(test_generator)

# Convert predictions to class labels
test_df["Predicted_Category"] = [class_labels[np.argmax(pred)] for pred in predictions]

# Display the first few rows of the table
print(test_df.head())

# Save the results to a CSV file
test_df.to_csv("sign_language_predictions.csv", index=False)


Found 1030 validated image filenames.
1030/1030 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step
    filename Predicted_Category
0  A2492.jpg                  A
1  A2527.jpg                  A
2  A2575.jpg                  A
3  A2621.jpg                  A
4  A2622.jpg                  A


In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the trained model
model = tf.keras.models.load_model("sign_language_model.h5")

# Define image size (same as training)
image_width, image_height = 128, 128
image_size = (image_width, image_height)

# Path to the test dataset folder
test_dir = r"C:\Users\mukes\Downloads\Sign-Language-detection-main\Sign-Language-detection-main\project1\testdatabase"

# Get the list of test filenames
test_filenames = os.listdir(test_dir)

# Creating a DataFrame for test data
test_df = pd.DataFrame({'filename': test_filenames})

# Create an ImageDataGenerator for testing (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_dir,
    x_col='filename',
    target_size=image_size,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# Get the class labels from the training data
class_labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Make predictions
predictions = model.predict(test_generator)

# Convert predictions to class labels
test_df["Predicted_Category"] = [class_labels[np.argmax(pred)] for pred in predictions]

# Display the first few rows of the table
print(test_df.head())

# Save the results to a CSV file
test_df.to_csv("sign_language_predictions.csv", index=False)


Found 1030 validated image filenames.


c:\Users\mukes\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1030/1030 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step
    filename Predicted_Category
0  A2492.jpg                  A
1  A2527.jpg                  A
2  A2575.jpg                  A
3  A2621.jpg                  A
4  A2622.jpg                  A
